In [1]:
# import sys
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install statsmodels
# !{sys.executable} -m pip install numpy

import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.iolib.summary2 import summary_col

In [2]:
data_panel_diff = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/data_election_crime_merged.csv')

In [3]:
data_panel_diff = pd.concat([data_panel_diff, pd.get_dummies(data_panel_diff['party'], prefix='party')],axis=1)
data_panel_diff = pd.concat([data_panel_diff, pd.get_dummies(data_panel_diff['state'], prefix='bl')],axis=1)

In [28]:
data_panel_diff.columns

Index(['Unnamed: 0', 'crime', 'city', 'law', 'date', 'background', 'suspects',
       'party', 'state', 'plz', 'city_id', 'bl_kuerzel', 'year', 'month',
       'day', 'cycle_1', 'cycle_2', 'cycle_3', 'crime_count',
       'Anzahl Bewerber', 'darunter Frauen', 'crime_count_party_lr',
       'crime_count_party', 'crime_count_lr', 'crime_count_city', 'party_lr',
       'Bewerber_city', 'Frauen_city', 'bewerber_hit_by_lr', 'fem_hit_by_lr',
       'bewerber_party_hit_by_lr', 'fem_party_hit_by_lr', 'p_female',
       'p_female_lr', 'p_female_city', 'p_female_party_lr', 'p_female_diff',
       'p_female_city_diff', 'p_female_lr_diff', 'p_female_party_lr_diff',
       'party_AfD', 'party_CDU', 'party_DIE LINKE', 'party_FDP', 'party_GRÜNE',
       'party_SPD', 'bl_BB', 'bl_BW', 'bl_SN', 'bl_TH', 'crime_count_no_afd',
       'crime_dummy'],
      dtype='object')

### Let's consider the standard measure of crime (against a party in a city)

In [5]:
data_panel_diff = data_panel_diff[data_panel_diff['cycle_1'] == 2014]
data_panel_diff = data_panel_diff.drop_duplicates(['city', 'party', 'plz', 'state', 'p_female_diff', 'crime_count_party'], ignore_index=True)
data_panel_diff

,Unnamed: 0,crime,city,law,date,background,suspects,party,state,plz,...,party_AfD,party_CDU,party_DIE LINKE,party_FDP,party_GRÜNE,party_SPD,bl_BB,bl_BW,bl_SN,bl_TH
0,17,Beleidigung,Stuttgart,185 StGB,2019-01-09,Links,1.0,AfD,BW,70173.0,...,1,0,0,0,0,0,0,1,0,0
1,19,NaN,Stuttgart,NaN,NaN,NaN,NaN,CDU,BW,70173.0,...,0,1,0,0,0,0,0,1,0,0
2,20,NaN,Stuttgart,NaN,NaN,NaN,NaN,DIE LINKE,BW,70173.0,...,0,0,1,0,0,0,0,1,0,0
3,21,NaN,Stuttgart,NaN,NaN,NaN,NaN,FDP,BW,70173.0,...,0,0,0,1,0,0,0,1,0,0
4,22,NaN,Stuttgart,NaN,NaN,NaN,NaN,GRÜNE,BW,70173.0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8967,20227,NaN,Pinnow,NaN,NaN,NaN,NaN,DIE LINKE,BB,16278.0,...,0,0,1,0,0,0,1,0,0,0
8968,20228,NaN,Pinnow,NaN,NaN,NaN,NaN,CDU,BB,16278.0,...,0,1,0,0,0,0,1,0,0,0
8969,20229,NaN,Pinnow,NaN,NaN,NaN,NaN,GRÜNE,BB,16278.0,...,0,0,0,0,1,0,1,0,0,0
8970,20233,NaN,Passow,NaN,NaN,NaN,NaN,CDU,BB,16306.0,...,0,1,0,0,0,0,1,0,0,0


In [6]:
data_panel_diff['p_female_diff'] = data_panel_diff['p_female_diff']*100

In [7]:
data_panel_diff['p_female_diff'].mean()

1.8683891691055903

In [8]:
data_panel_diff[['p_female_diff', 'city', 'crime_count_party']]

,p_female_diff,city,crime_count_party
0,6.666667,Stuttgart,14.0
1,-5.000000,Stuttgart,1.0
2,1.666667,Stuttgart,0.0
3,-1.666667,Stuttgart,4.0
4,0.000000,Stuttgart,1.0
...,...,...,...
8967,20.000000,Pinnow,0.0
8968,5.555556,Pinnow,0.0
8969,0.000000,Pinnow,0.0
8970,0.000000,Passow,0.0


In [9]:
print("Conditional means based on party (no restriction to wether a crime has happened or whether the party stood twice)")
data_panel_diff.groupby(['party'], as_index=False).mean()[['party', 'crime_count_party', 'p_female_diff']], len(data_panel_diff[~data_panel_diff['p_female_diff'].isna()])

Conditional means based on party (no restriction to wether a crime has happened or whether the party stood twice)


(       party  crime_count_party  p_female_diff
 0        AfD           0.348513       8.293668
 1        CDU           0.083969       1.369342
 2  DIE LINKE           0.000000       2.868536
 3        FDP           0.034605       1.608198
 4      GRÜNE           0.075806       3.159392
 5        SPD           0.163842       1.339538,
 3843)

In [10]:
# Drop missing values
data_panel_diff.dropna(subset=['p_female_diff', 'crime_count_party'], inplace=True)
print("Conditional means based on party when they stood for office twice")
data_panel_diff.groupby(['party'], as_index=False).mean()[['party', 'crime_count_party', 'p_female_diff']], len(data_panel_diff[~data_panel_diff['p_female_diff'].isna()])

Conditional means based on party when they stood for office twice


(       party  crime_count_party  p_female_diff
 0        AfD           5.583333       8.293668
 1        CDU           0.106242       1.369342
 2  DIE LINKE           0.000000       2.868536
 3        FDP           0.121951       1.608198
 4      GRÜNE           0.217507       3.159392
 5        SPD           0.266667       1.339538,
 3843)

In [11]:
X = data_panel_diff['crime_count_party']
y = data_panel_diff['p_female_diff']
X = sm.add_constant(X)
model1 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party_lr']})
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     5.800
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.251
Time:                        23:02:11   Log-Likelihood:                -16512.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3841   BIC:                         3.304e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.8567      0.278      6.687      0.000       1.313       2.401
crime_count_party     0.0553      0.023      2.408      0.016       0.010       0.100
==============================================================================
Omnibus:                      625.666   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5603.544
Skew:                           0.502   Prob(JB):                         0.00
Kurtosis:                       8.830   Cond. No.                         2.09
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [12]:
model2 = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party']})
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     3.065
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.140
Time:                        23:02:11   Log-Likelihood:                -16512.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3841   BIC:                         3.304e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.8567      0.346      5.364      0.000       1.178       2.535
crime_count_party     0.0553      0.032      1.751      0.080      -0.007       0.117
==============================================================================
Omnibus:                      625.666   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5603.544
Skew:                           0.502   Prob(JB):                         0.00
Kurtosis:                       8.830   Cond. No.                         2.09
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [13]:
print(summary_col([model1,model2]))
# A increase by one crime against a party is associated with an 0.0005% higher share of females in the following period. 


                  p_female_diff I p_female_diff II
--------------------------------------------------
const             1.8567          1.8567          
                  (0.2776)        (0.3461)        
crime_count_party 0.0553          0.0553          
                  (0.0229)        (0.0316)        
R-squared         0.0000          0.0000          
R-squared Adj.    -0.0002         -0.0002         
Standard errors in parentheses.


In [14]:
data_panel_diff2 = data_panel_diff.drop_duplicates(['city', 'plz', 'state', 'p_female_city_diff', 'crime_count_city'], ignore_index=True)
data_panel_diff2.dropna(subset=['p_female_city_diff', 'crime_count_city'], inplace=True)

X2 = data_panel_diff2["crime_count_city"]
y = data_panel_diff2["p_female_city_diff"]
X2 = sm.add_constant(X2)
model4 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff2['party_lr']})
model4.summary()

<ipython-input-14-311ba1b77476>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_panel_diff2.dropna(subset=['p_female_city_diff', 'crime_count_city'], inplace=True)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     p_female_city_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2234
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.719
Time:                        23:02:11   Log-Likelihood:                 1144.8
No. Observations:                1666   AIC:                            -2286.
Df Residuals:                    1664   BIC:                            -2275.
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0037      0.006      0.649      0.516      -0.007       0.015
crime_count_city  2.748e-05   5.81e-05      0.473      0.636   -8.65e-05       0.000
==============================================================================
Omnibus:                      289.746   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2742.155
Skew:                          -0.517   Prob(JB):                         0.00
Kurtosis:                       9.199   Cond. No.                         5.69
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [15]:
X2 = data_panel_diff2["crime_count_city"]
y = data_panel_diff2["p_female_city_diff"]
X2 = sm.add_constant(X2)
model4 = sm.OLS(y, X2).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff2['party']})
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     p_female_city_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.02033
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.892
Time:                        23:02:11   Log-Likelihood:                 1144.8
No. Observations:                1666   AIC:                            -2286.
Df Residuals:                    1664   BIC:                            -2275.
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0037      0.004      0.889      0.374      -0.004       0.012
crime_count_city  2.748e-05      0.000      0.143      0.887      -0.000       0.000
==============================================================================
Omnibus:                      289.746   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2742.155
Skew:                          -0.517   Prob(JB):                         0.00
Kurtosis:                       9.199   Cond. No.                         5.69
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [16]:
data_panel_lr = data_panel_diff.drop_duplicates(['city', 'party', 'plz', 'state', 'p_female_party_lr', 'crime_count_party_lr'], ignore_index=True)
data_panel_lr.dropna(subset=['p_female_party_lr', 'crime_count_party_lr'], inplace=True)

X2 = data_panel_lr["crime_count_party_lr"]
y = data_panel_lr["p_female_party_lr"]
X2 = sm.add_constant(X2)
model4 = sm.OLS(y, X2).fit(cov_type='HC3')
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      p_female_party_lr   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.3475
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.562
Time:                        23:02:11   Log-Likelihood:                 10.978
No. Observations:                  24   AIC:                            -17.96
Df Residuals:                      22   BIC:                            -15.60
Df Model:                           1                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.2614      0.040      6.609      0.000       0.184       0.339
crime_count_party_lr    -0.0013      0.002     -0.589      0.556      -0.006       0.003
==============================================================================
Omnibus:                        1.365   Durbin-Watson:                   1.578
Prob(Omnibus):                  0.505   Jarque-Bera (JB):                0.938
Skew:                          -0.127   Prob(JB):                        0.626
Kurtosis:                       2.066   Cond. No.                         23.5
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [17]:
%run estout_func.ipynb

In [18]:
estout_ols(modellist=[model1, model2], modellist_str = ["model1","model2"], y='Share Female Candidates', caption="Basic Regression", label="Basic Regression", list_regression_sets=[X,X], p_values=True)

\begin{table}[htbp] \caption{Basic Regression \label{Basic Regression}}
\resizebox{0.9\textwidth}{!}{ \centering
\begin{tabular}{lcc} \hline
  & (I) & (II) \\ 
Dependent Variable & \multicolumn{2}{c}{Share Female Candidates} \\ \hline \vspace{4pt} 
& \begin{footnotesize}\end{footnotesize} & \begin{footnotesize}\end{footnotesize}  \\ 
const & 1.8567*** & 1.8567*** \\ 
 \vspace{4pt} & \begin{footnotesize}(0.2776) \end{footnotesize} & \begin{footnotesize}(0.3461) \end{footnotesize} \\ 
crime_count_party & 0.0553** & 0.0553* \\ 
 \vspace{4pt} & \begin{footnotesize}(0.0229) \end{footnotesize} & \begin{footnotesize}(0.0316) \end{footnotesize} \\ 
Observations & 3843 & 3843 \\ 
Adj. $R^2$ & -0.0002 & -0.0002 \\ 
F-statistic & 5.799998655668696 & 3.064992199959257 \\ 
Mean Squared Error & 49.72543093166314 & 49.72543093166314 \\ \hline
\multicolumn{3}{c}{\begin{footnotesize} model1 has cluster,  model2 has cluster, standard errors in parentheses. \end{footnotesize}} \\ 
\multicolumn{3}{c}{\beg

In [19]:
X3 = data_panel_diff[["crime_count_party", "party_GRÜNE", "party_SPD", "party_CDU", "party_FDP", "party_AfD", 'bl_BB', 'bl_SN', 'bl_TH']]
y2 = data_panel_diff["p_female_diff"]
X3 = sm.add_constant(X3)
model6 = sm.OLS(y2, X3).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party_lr']})
model6.summary()

C:\Users\Marius\Anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 1
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                   0.01206
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.930
Time:                        23:02:11   Log-Likelihood:                -16499.
No. Observations:                3843   AIC:                         3.302e+04
Df Residuals:                    3833   BIC:                         3.308e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.5511      0.395      3.922      0.000       0.776       2.326
crime_count_party    -0.0340      0.048     -0.706      0.480      -0.129       0.060
party_GRÜNE           0.8354      0.218      3.836      0.000       0.409       1.262
party_SPD            -0.9156      0.233     -3.926      0.000      -1.373      -0.458
party_CDU            -0.8280      0.163     -5.085      0.000      -1.147      -0.509
party_FDP            -0.9452      0.154     -6.134      0.000      -1.247      -0.643
party_AfD             4.9353      0.001   4527.355      0.000       4.933       4.937
bl_BB                 2.9195      0.131     22.255      0.000       2.662       3.177
bl_SN                 0.8280      1.004      0.824      0.410      -1.141       2.797
bl_TH                 0.0069      0.008      0.816      0.414      -0.010       0.023
==============================================================================
Omnibus:                      555.858   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5300.127
Skew:                           0.370   Prob(JB):                         0.00
Kurtosis:                       8.705   Cond. No.                         20.9
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [20]:
X3 = data_panel_diff[["crime_count_party", "party_GRÜNE", "party_SPD", "party_CDU", "party_FDP", "party_DIE LINKE", 'bl_SN', 'bl_TH', 'bl_BB']]
y2 = data_panel_diff["p_female_diff"]
X3 = sm.add_constant(X3)
model7 = sm.OLS(y2, X3).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party']})
model7.summary()

C:\Users\Marius\Anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 4
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     269.5
Date:                Fri, 24 Sep 2021   Prob (F-statistic):           5.05e-06
Time:                        23:02:12   Log-Likelihood:                -16499.
No. Observations:                3843   AIC:                         3.302e+04
Df Residuals:                    3833   BIC:                         3.308e+04
Df Model:                           9                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 6.4864      0.601     10.788      0.000       5.308       7.665
crime_count_party    -0.0340      0.045     -0.753      0.452      -0.123       0.055
party_GRÜNE          -4.0999      0.406    -10.089      0.000      -4.896      -3.303
party_SPD            -5.8509      0.423    -13.831      0.000      -6.680      -5.022
party_CDU            -5.7634      0.432    -13.341      0.000      -6.610      -4.917
party_FDP            -5.8805      0.372    -15.798      0.000      -6.610      -5.151
party_DIE LINKE      -4.9353      0.349    -14.154      0.000      -5.619      -4.252
bl_SN                 0.8280      0.736      1.125      0.261      -0.615       2.271
bl_TH                 0.0069      0.464      0.015      0.988      -0.902       0.916
bl_BB                 2.9195      0.542      5.383      0.000       1.856       3.983
==============================================================================
Omnibus:                      555.858   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5300.127
Skew:                           0.370   Prob(JB):                         0.00
Kurtosis:                       8.705   Cond. No.                         48.3
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [21]:
data_panel_diff["crime_count_no_afd"] = data_panel_diff["crime_count_party"]
data_panel_diff.loc[(data_panel_diff["party_AfD"]==1), 'crime_count_no_afd'] = 0
X4 = data_panel_diff[["crime_count_no_afd", "party_SPD", "party_GRÜNE", "party_CDU", "party_AfD", 'bl_SN', 'bl_TH', 'bl_BB']]
y3 = data_panel_diff["p_female_diff"]
X4 = sm.add_constant(X4)
model8 = sm.OLS(y3, X4).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['party_lr']})
model8.summary()

C:\Users\Marius\Anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 1
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                    0.4819
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.614
Time:                        23:02:42   Log-Likelihood:                -16499.
No. Observations:                3843   AIC:                         3.302e+04
Df Residuals:                    3834   BIC:                         3.307e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  1.1297      0.147      7.665      0.000       0.841       1.419
crime_count_no_afd     0.0906      0.049      1.852      0.064      -0.005       0.187
party_SPD             -0.5511      0.293     -1.882      0.060      -1.125       0.023
party_GRÜNE            1.2055      0.308      3.915      0.000       0.602       1.809
party_CDU             -0.4484      0.361     -1.243      0.214      -1.156       0.259
party_AfD              5.1237      0.240     21.345      0.000       4.653       5.594
bl_SN                  0.8820      0.975      0.904      0.366      -1.030       2.794
bl_TH                  0.0525      0.023      2.288      0.022       0.008       0.097
bl_BB                  2.9704      0.177     16.793      0.000       2.624       3.317
==============================================================================
Omnibus:                      556.767   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5289.838
Skew:                           0.373   Prob(JB):                         0.00
Kurtosis:                       8.699   Cond. No.                         11.1
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [25]:
data_panel_diff["crime_dummy"] = 0
data_panel_diff.loc[(data_panel_diff["crime_count_party"]!=0), 'crime_dummy'] = 1
X3 = data_panel_diff[["crime_dummy", "party_SPD", "party_GRÜNE", "party_CDU", "party_AfD", 'bl_SN', 'bl_TH', 'bl_BB']]
y = data_panel_diff["p_female_diff"]
X3 = sm.add_constant(X3)
model9 = sm.OLS(y, X3).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['state']})
model9.summary()

C:\Users\Marius\Anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 3
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.50
Date:                Fri, 24 Sep 2021   Prob (F-statistic):             0.0424
Time:                        23:11:12   Log-Likelihood:                -16499.
No. Observations:                3843   AIC:                         3.302e+04
Df Residuals:                    3834   BIC:                         3.307e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.1191      0.474      2.363      0.018       0.191       2.047
crime_dummy    -0.4018      0.387     -1.038      0.299      -1.160       0.357
party_SPD      -0.4928      0.514     -0.958      0.338      -1.501       0.515
party_GRÜNE     1.2627      0.973      1.298      0.194      -0.645       3.170
party_CDU      -0.4238      0.698     -0.607      0.544      -1.792       0.944
party_AfD       5.2947      0.944      5.607      0.000       3.444       7.145
bl_SN           0.9110      0.120      7.617      0.000       0.677       1.145
bl_TH           0.0847      0.101      0.837      0.402      -0.114       0.283
bl_BB           2.9987      0.129     23.170      0.000       2.745       3.252
==============================================================================
Omnibus:                      554.567   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5281.799
Skew:                           0.368   Prob(JB):                         0.00
Kurtosis:                       8.696   Cond. No.                         11.2
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [27]:
X4 = data_panel_diff[["crime_dummy"]]
y = data_panel_diff["p_female_diff"]
X4 = sm.add_constant(X4)
model10 = sm.OLS(y, X4).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff['state']})
model10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.2274
Date:                Fri, 24 Sep 2021   Prob (F-statistic):              0.666
Time:                        23:21:40   Log-Likelihood:                -16512.
No. Observations:                3843   AIC:                         3.303e+04
Df Residuals:                    3841   BIC:                         3.304e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.8479      0.700      2.640      0.008       0.476       3.220
crime_dummy     0.3122      0.655      0.477      0.633      -0.971       1.595
==============================================================================
Omnibus:                      625.859   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5603.483
Skew:                           0.502   Prob(JB):                         0.00
Kurtosis:                       8.830   Cond. No.                         4.06
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [71]:
data_panel_diff_balanced_noafd = data_panel_diff[data_panel_diff['party_FDP'] != 1]
X4 = data_panel_diff_balanced_noafd[["crime_count_party", "party_SPD", "party_GRÜNE", "party_CDU", "party_AfD", 'bl_SN', 'bl_TH', 'bl_BB']]
y3 = data_panel_diff_balanced_noafd["p_female_diff"]
X4 = sm.add_constant(X4)
model6 = sm.OLS(y3, X4).fit(cov_type='cluster', cov_kwds={'groups': data_panel_diff_balanced_noafd['party']})
model6.summary()

C:\Users\Marius\Anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 4
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          p_female_diff   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     241.6
Date:                Sun, 12 Sep 2021   Prob (F-statistic):           5.08e-05
Time:                        21:59:08   Log-Likelihood:                -14895.
No. Observations:                3474   AIC:                         2.981e+04
Df Residuals:                    3465   BIC:                         2.986e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.6834      0.462      3.645      0.000       0.778       2.589
crime_count_party    -0.0429      0.041     -1.033      0.301      -0.124       0.038
party_SPD            -0.9760      0.251     -3.896      0.000      -1.467      -0.485
party_GRÜNE           0.7743      0.233      3.318      0.001       0.317       1.232
party_CDU            -0.8818      0.210     -4.195      0.000      -1.294      -0.470
party_AfD             4.9988      0.353     14.143      0.000       4.306       5.692
bl_SN                 0.7568      0.798      0.948      0.343      -0.808       2.321
bl_TH                -0.1636      0.440     -0.372      0.710      -1.026       0.699
bl_BB                 2.7091      0.529      5.120      0.000       1.672       3.746
==============================================================================
Omnibus:                      507.411   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5040.143
Skew:                           0.359   Prob(JB):                         0.00
Kurtosis:                       8.857   Cond. No.                         20.7
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""